In [1]:
import sys
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
sys.path.append("../gotmtool")
from gotmtool import *

## Create a model
Create a model with environment file `../../.gotm_env.yaml`, which is created by `gotm_env_init.py`. 

In [2]:
m = Model(name='SD_1D', environ='../gotmtool/.gotm_env.yaml')

Take a look at what are defined in the environment file.

In [3]:
for key in m.environ:
    print('{:>15s}: {}'.format(key, m.environ[key]) )

   gotmdir_code: /Users/qingli/work/saildrone_southern_ocean/gotm/code
   gotmdir_data: /Users/qingli/work/saildrone_southern_ocean/gotm/data
  gotmdir_build: /Users/qingli/work/saildrone_southern_ocean/gotm/build
    gotmdir_exe: /Users/qingli/work/saildrone_southern_ocean/gotm/exe
    gotmdir_run: /Users/qingli/work/saildrone_southern_ocean/gotm/run
 gotmdir_figure: /Users/qingli/work/saildrone_southern_ocean/gotm/figure
   gotmdir_tool: /Users/qingli/work/saildrone_southern_ocean/gotmtool


## Build the model

In [4]:
%%time
m.build()

CPU times: user 968 µs, sys: 6.81 ms, total: 7.77 ms
Wall time: 88.7 ms


## Configuration
Initialize the GOTM configuration

In [6]:
cfgfile = m.environ['gotmdir_run']+'/SD_1D/gotm.yaml'
if os.path.exists(cfgfile):
    print('loading configure file...')
    cfg = yaml_load(filename=cfgfile)
else:
    cfg = m.init_config(filename=cfgfile)

loading configure file...


Update the configuration

In [7]:
# setup
title = 'Constant forcing'
nlev = 400
cfg['title'] = title
cfg['location']['name'] = 'Idealized'
cfg['location']['latitude'] = -55.
cfg['location']['longitude'] = -145.
cfg['location']['depth'] = 200
cfg['time']['start'] = '2019-02-07 00:00:00'
cfg['time']['stop']  = '2019-02-12 00:00:00'
cfg['time']['dt']    = 60.0
cfg['grid']['nlev']  = nlev

# output
cfg['output'] = {}
cfg['output']['gotm_out'] = {}
cfg['output']['gotm_out']['use'] = True
cfg['output']['gotm_out']['title'] = title
cfg['output']['gotm_out']['k1_stop'] = nlev+1
cfg['output']['gotm_out']['k_stop'] = nlev
cfg['output']['gotm_out']['time_unit'] = 'dt'
cfg['output']['gotm_out']['time_step'] = 10
cfg['output']['gotm_out']['variables'] = [{}]
cfg['output']['gotm_out']['variables'][0]['source'] = '*'

# forcing
datadir = '/Users/qingli/work/saildrone_southern_ocean/SD_1D_cases'
cfg['temperature']['method'] = 'file'
cfg['salinity']['method'] = 'file'
cfg['surface']['fluxes']['heat']['method'] = 'constant'
cfg['surface']['fluxes']['tx']['method'] = 'constant'
cfg['surface']['fluxes']['ty']['method'] = 'constant'
cfg['surface']['fluxes']['ty']['constant_value'] = 0.0
cfg['surface']['swr']['method'] = 'file'
cfg['surface']['swr']['file'] = datadir+'/swr_era5.dat'
cfg['surface']['precip']['method'] = 'constant'
cfg['waves']['stokes_drift']['us']['method'] = 'off'
cfg['waves']['stokes_drift']['vs']['method'] = 'off'

# EOS -- use linear
cfg['eq_state']['form'] = 'full-pot'

# water type (Jerlov IB)
cfg['light_extinction']['method'] = 'jerlov-ib'

In [8]:
cfg['turbulence']['turb_method'] = 'second_order'
cfg['turbulence']['tke_method'] = 'tke'
cfg['turbulence']['len_scale_method'] = 'gls'
cfg['turbulence']['scnd']['method'] =  'weak_eq_kb_eq'
cfg['turbulence']['scnd']['scnd_coeff'] =  'canuto-a'
cfg['turbulence']['turb_param']['length_lim'] = 'false'
cfg['turbulence']['turb_param']['compute_c3'] = 'true'
cfg['turbulence']['turb_param']['Ri_st'] = 0.25
cfg['turbulence']['generic']['gen_m'] = 1.5 
cfg['turbulence']['generic']['gen_n'] = -1.0
cfg['turbulence']['generic']['gen_p'] = 3.0 
cfg['turbulence']['generic']['cpsi1'] = 1.44
cfg['turbulence']['generic']['cpsi2'] = 1.92
cfg['turbulence']['generic']['cpsi3minus'] = -0.63
cfg['turbulence']['generic']['cpsi3plus'] = 1.0 
cfg['turbulence']['generic']['sig_kpsi'] = 1.0 
cfg['turbulence']['generic']['sig_psi'] = 1.3

In [9]:
# collect the configurations and the labels of the two runs
cfgs = []
labels = []

In [10]:
hfpmin = {
    'C1': -25.3,
    'C4': -3.0,
    'C5': -7.8,
    'C6': -9.0,
    'C7': -8.6,
    'C8': -9.0,
    'P1': -42.9,
    'P2': -104.6,
    'P3': -49.6,
    'P4': -36.4,
    'P5': -20.9,
    'P6': -18.1,
    'P7': -15.5,
    'P8': -55.7,
    }
hfpmax = {
    'C1': 16.7,
    'C4': 3.4,
    'C5': 8.8,
    'C6': 9.3,
    'C7': 8.3,
    'C8': 6.1,
    'P1': 29.7,
    'P2': 55.6,
    'P3': 44.4,
    'P4': 40.8,
    'P5': 23.2,
    'P6': 16.8,
    'P7': 14.5,
    'P8': 55.8,
    }
hfref = {
    'C1': -68.9,
    'C4': -26.2,
    'C5': -56.4,
    'C6': -47.1,
    'C7': -109.0,
    'C8': -45.7,
    'P1': -113.2,
    'P2': -209.1,
    'P3': -218.2,
    'P4': -170.8,
    'P5': -134.3,
    'P6': -73.9,
    'P7': -30.9,
    'P8': -210.8,
    }
taudat = {
    'C1': 0.2,
    'C4': 0.07,
    'C5': 0.07,
    'C6': 0.08,
    'C7': 0.03,
    'C8': 0.26,
    'P1': 0.49,
    'P2': 0.18,
    'P3': 0.45,
    'P4': 0.4,
    'P5': 0.22,
    'P6': 0.14,
    'P7': 0.13,
    'P8': 0.53,
    }
argo_dates = {
    'C1': 33,
    'C4': 47,
    'C5': 52,
    'C6': 55,
    'C7': 62,
    'C8': 69,
    'P1': 34,
    'P2': 36,
    'P3': 40,
    'P4': 41,
    'P5': 43,
    'P6': 52,
    'P7': 54,
    'P8': 58,
    }
year = 2019
startdates = {}
stopdates = {}
for key in argo_dates.keys():
    dttime_start = pd.to_datetime(year*1000 + argo_dates[key], format='%Y%j')
    startdates[key] = dttime_start.strftime('%Y-%m-%d %H:%M:%S')
    dttime_stop = pd.to_datetime(year*1000 + argo_dates[key]+5, format='%Y%j')
    stopdates[key] = dttime_stop.strftime('%Y-%m-%d %H:%M:%S')
hfdat = {}
for key in hfref.keys():
    hfdat[key] = [hfref[key], hfref[key]+hfpmax[key], hfref[key]+hfpmin[key]]
clabels = ['ref', 'pos', 'neg']

for key in hfref.keys():
    for i, ckey in enumerate(clabels):
        rlabel = '_'.join([key, ckey])
        print(rlabel)
        cfg['time']['start'] = startdates[key]
        cfg['time']['stop']  = stopdates[key]
        cfg['turbulence']['turb_method'] = 'second_order' 
        cfg['temperature']['file'] = datadir+'/t_prof_{}.dat'.format(key)
        cfg['salinity']['file'] = datadir+'/s_prof_{}.dat'.format(key)
        cfg['surface']['fluxes']['heat']['constant_value'] = hfdat[key][i]
        cfg['surface']['fluxes']['tx']['constant_value'] = taudat[key]
        cfgs.append(copy.deepcopy(cfg))
        labels.append(rlabel+'_GLS-C01A')
        cfg['turbulence']['turb_method'] = 'cvmix'
        cfg['cvmix']['surface_layer']['kpp']['langmuir_method'] = 'none'
        cfgs.append(copy.deepcopy(cfg))
        labels.append(rlabel+'_KPP-CVMix')

C1_ref
C1_pos
C1_neg
C4_ref
C4_pos
C4_neg
C5_ref
C5_pos
C5_neg
C6_ref
C6_pos
C6_neg
C7_ref
C7_pos
C7_neg
C8_ref
C8_pos
C8_neg
P1_ref
P1_pos
P1_neg
P2_ref
P2_pos
P2_neg
P3_ref
P3_pos
P3_neg
P4_ref
P4_pos
P4_neg
P5_ref
P5_pos
P5_neg
P6_ref
P6_pos
P6_neg
P7_ref
P7_pos
P7_neg
P8_ref
P8_pos
P8_neg


## Run the model

In [11]:
%%time
sims = m.run_batch(configs=cfgs, labels=labels, nproc=4)

CPU times: user 89.9 ms, sys: 40.8 ms, total: 131 ms
Wall time: 23.8 s
